In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import pickle
import pylab
import numpy as np
import scipy
import torch.optim as optim
import pandas as pd
import torchvision.datasets as datasets

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import shapiro, normaltest

from torchvision import transforms

from copy import deepcopy

# Local imports
from local_models import *
from helper_functions import *
from piece_hurdle_model import *
from optimize_explanations import *
from evaluation_metrics import *

from IPython.display import Image
import numpy as np

In [2]:
def load_fashion_dataloaders():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_set = torchvision.datasets.FashionMNIST(
        root='./data/after_anon_review',
        train=True,
        download=True,
        transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=1,
        shuffle=False
    )

    test_set = torchvision.datasets.FashionMNIST(
        root='./data/after_anon_review',
        train=False,
        download=True,
        transform=transform
    )
    test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=1,
        shuffle=False
    )

    return train_loader, test_loader

In [3]:
G, cnn = load_models(CNN, Generator)

#train_loader, test_loader = load_dataloaders()
train_loader, test_loader = load_fashion_dataloaders()
X_train, y_train, X_test, y_test = get_MNIST_data(datasets)

In [4]:
def get_FashionMNIST_data(datasets):
    fashion_mnist_trainset = datasets.FashionMNIST(root='./data/fashion_mnist_train', train=True, download=True, transform=None)
    fashion_mnist_testset = datasets.FashionMNIST(root='./data/fashion_mnist_test', train=False, download=True, transform=None)
    X_train = fashion_mnist_trainset.data
    y_train = fashion_mnist_trainset.targets
    X_test = fashion_mnist_testset.data
    y_test = fashion_mnist_testset.targets
    return X_train, y_train, X_test, y_test


In [5]:
def return_feature_contribution_data(data_loader, cnn, num_classes=10):

    """
    Return feature contribution data for each class based on the given data loader and CNN model.
    
    Args:
    - data_loader: DataLoader object containing the dataset to analyze.
    - cnn: CNN model used for feature extraction.
    - num_classes: Number of classes in the dataset. Default is 10.
    
    Returns:
    - pred_idx: A dictionary containing feature contribution data for each class.
                Keys represent class names, and values are lists of feature contributions.
    """
    
    full_data = dict()
    pred_idx = dict() 

    for class_name in list(range(num_classes)):
        pred_idx[class_name] = list()
        
    for i, data in enumerate(data_loader):
        # print progress
        if i % 10000 == 0:
            print(  100 * round(i / len(data_loader), 2), "% complete..."  )     
        image, label = data
        label = int(label.detach().numpy())
        acts = cnn(image)[1][0].detach().numpy()
        pred = int(torch.argmax(  cnn(image)[0]  ).detach().numpy()) 
        pred_idx[pred].append(acts.tolist())
                
    return pred_idx

In [6]:
# collected_data is a dictionary where each key represents a class, and the corresponding value is a list containing activations for that class.
collected_data = return_feature_contribution_data(train_loader, cnn)
dist_data = {}

# Assuming num_classes is the number of classes
num_classes = 10

# Create an empty list for each class
for class_name in range(num_classes):
    dist_data[class_name] = {'activations': []}

# Fill data from pred_idx_train into dist_data
for class_name, activations_list in collected_data.items():
    # Convert activations_list to a numpy array
    activations_array = np.array(activations_list)
    # Store activations_array into the corresponding class in dist_data
    dist_data[class_name]['activations'] = activations_array

0.0 % complete...


C:\Users\13478\AppData\Local\Temp\ipykernel_18932\2353536552.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(label.detach().numpy())


17.0 % complete...
33.0 % complete...
50.0 % complete...
67.0 % complete...
83.0 % complete...


In [10]:
with open('collected_fashion.pickle', 'wb') as handle:
    pickle.dump(dist_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("save pickle ")

NameError: name 'dist_data' is not defined